In [3]:
# !pip install transformers
# !pip install tokenizers
!pip install tensorboard==2.1.0

     |████████████████████████████████| 3.8MB 40.8MB/s 
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.0 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1


In [0]:
import regex as re
def basicPreprocess(text):
  try:
    processed_text = text.lower()
    processed_text = re.sub(r'\W +', ' ', processed_text)
  except Exception as e:
    print("Exception:",e,",on text:", text)
    return None
  return processed_text

In [0]:
import pandas as pd
from tqdm import tqdm

In [0]:
data = pd.read_csv("/content/drive/My Drive/Clothing NLP/descriptions.txt",header=None,encoding='utf-8')
data.columns = ['description']

In [0]:
data["description"] = data["description"].apply(basicPreprocess).dropna() #ignor

In [0]:
data = data["description"]
data = data.replace("\n"," ")

In [0]:
txt_files_dir = "/tmp/text_split"
!mkdir {txt_files_dir}

In [0]:
import os

In [0]:
i=0
for row in tqdm(data.to_list()):
  file_name = os.path.join(txt_files_dir, str(i)+'.txt')
  try:
    f = open(file_name, 'w')
    f.write(row)
    f.close()
  except Exception as e:  #catch exceptions(for eg. empty rows)
    print(row, e) 
  i+=1

100%|██████████| 113497/113497 [00:05<00:00, 20625.05it/s]


In [0]:
!pip install git+https://github.com/huggingface/tokenizers

  Cloning https://github.com/huggingface/tokenizers to /tmp/pip-req-build-_xn4taf4
  Running command git clone -q https://github.com/huggingface/tokenizers /tmp/pip-req-build-_xn4taf4
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [0]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3m4pyw0j
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-3m4pyw0j
     |████████████████████████████████| 3.8MB 19.6MB/s 
     |████████████████████████████████| 1.0MB 20.2MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
  Created wheel for transformers: filename=transformers-2.8.0-cp36-none-any.whl size=595751 sha256=1db25720d9b7de58df2381109b0c41b331a3ed36355b1ff0ac67f83bc261db00
  Stored in directory: /tmp/pip-ephem-wheel-cache-dee4yut8/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=b95605ef8ae5fea087ca90bc0054fd6cc0f853c424f7c7738b7573ab553498c3
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built transformers sacremoses


In [0]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


paths = ['/content/drive/My Drive/Clothing NLP/descriptions.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

vocab_size=30000
# Customize training
tokenizer.train(files=paths, vocab_size=vocab_size, min_frequency=5, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [0]:
tokenizer.save(".", "clothberta")

['./clothberta-vocab.json', './clothberta-merges.txt']

In [0]:
lm_data_dir = "/tmp/lm_data"
!mkdir {lm_data_dir}

In [0]:

train_split = 0.9
train_data_size = int(len(data)*train_split)

with open(os.path.join(lm_data_dir,'train.txt') , 'w') as f:
    for item in data[:train_data_size].tolist():
        f.write("%s\n" % item)

with open(os.path.join(lm_data_dir,'eval.txt') , 'w') as f:
    for item in data[train_data_size:].tolist():
        f.write("%s\n" % item)

In [0]:
!mkdir /content/models
!mkdir /content/models/smallBERTa

In [0]:
tokenizer.save("/content/models/smallBERTa", "smallBERTa")

['/content/models/smallBERTa/smallBERTa-vocab.json',
 '/content/models/smallBERTa/smallBERTa-merges.txt']

In [0]:
!mv /content/models/smallBERTa/smallBERTa-vocab.json /content/models/smallBERTa/vocab.json
!mv /content/models/smallBERTa/smallBERTa-merges.txt /content/models/smallBERTa/merges.txt

In [0]:
train_path = os.path.join(lm_data_dir,"train.txt")
eval_path = os.path.join(lm_data_dir,"eval.txt")

In [0]:
import json
config = {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "num_attention_heads": 1,
  "num_hidden_layers": 1,
  "vocab_size": vocab_size,
  "intermediate_size": 256,
  "max_position_embeddings": 256,
  "model_type": 'roberta'
}
with open("/content/models/smallBERTa/config.json", 'w') as fp:
    json.dump(config, fp)

In [0]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 26170, done.
remote: Total 26170 (delta 0), reused 0 (delta 0), pack-reused 26170
Receiving objects: 100% (26170/26170), 15.31 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (18286/18286), done.


In [1]:
!nvidia-smi

Sun May  3 17:24:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
os.environ["train_path"] = train_path
os.environ["eval_path"] = eval_path
os.environ["CUDA_LAUNCH_BLOCKING"]='1'  #Makes for easier debugging (just in case)
weights_dir = "/content/models/smallBERTa/weights"
!mkdir {weights_dir}

NameError: ignored

In [0]:
cmd = '''python /content/transformers/examples/run_language_modeling.py --output_dir {0}  \
    --model_type roberta \
    --mlm \
    --train_data_file {1} \
    --eval_data_file {2} \
    --config_name /content/models/smallBERTa \
    --tokenizer_name /content/models/smallBERTa \
    --do_train \
    --line_by_line \
    --overwrite_output_dir \
    --do_eval \
    --block_size 256 \
    --learning_rate 1e-4 \
    --num_train_epochs 5 \
    --save_total_limit 2 \
    --save_steps 2000 \
    --logging_steps 500 \
    --per_gpu_eval_batch_size 32 \
    --per_gpu_train_batch_size 4 \
    --evaluate_during_training \
    --seed 42 \
    '''.format(weights_dir, train_path, eval_path)

In [0]:
!{cmd}

流式输出内容被截断，只能显示最后 5000 行内容。
Evaluation: 100% 426/426 [00:06<00:00, 69.65it/s]

{"eval_loss": 5.8693884303312345, "learning_rate": 1.102514506769826e-05, "loss": 4.598767858505249, "step": 115000}

Epoch:  80% 4/5 [49:54<11:10, 670.89s/it]
Iteration:  45% 11602/25850 [05:07<57:42,  4.12it/s]
Iteration:  45% 11610/25850 [05:07<41:17,  5.75it/s]
Iteration:  45% 11618/25850 [05:07<29:49,  7.95it/s]
Iteration:  45% 11625/25850 [05:07<21:54, 10.83it/s]
Iteration:  45% 11633/25850 [05:07<16:14, 14.58it/s]
Iteration:  45% 11641/25850 [05:08<12:18, 19.25it/s]
Iteration:  45% 11649/25850 [05:08<09:41, 24.43it/s]
Iteration:  45% 11657/25850 [05:08<07:43, 30.65it/s]
Iteration:  45% 11665/25850 [05:08<06:25, 36.79it/s]
Iteration:  45% 11673/25850 [05:08<05:28, 43.16it/s]
Iteration:  45% 11681/25850 [05:08<04:50, 48.83it/s]
Iteration:  45% 11689/25850 [05:08<04:19, 54.50it/s]
Iteration:  45% 11697/25850 [05:08<03:58, 59.35it/s]
Iteration:  45% 11705/25850 [05:08<03:45, 62.74it/s]
Iteration:  45% 1171

In [0]:
!tensorboard dev upload --logdir /content/runs

2020-05-03 07:36:39.045282: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

/content/runs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3

In [0]:
!zip -r /content/models.zip /content/models
from google.colab import files
files.download("/content/models.zip")

  adding: content/models/ (stored 0%)
  adding: content/models/smallBERTa/ (stored 0%)
  adding: content/models/smallBERTa/smallBERTa-vocab.json (deflated 59%)
  adding: content/models/smallBERTa/smallBERTa-merges.txt (deflated 54%)
